In [1]:
##################################################################
#《Python机器学习及实践：从零开始通往Kaggle竞赛之路（2024年度版）》开源代码
#-----------------------------------------------------------------
#                 @章节号：6.8.2（Transformer模型的TensorFlow实践）                           
#                 @作者：范淼、徐晟桐 
#                 @购书链接：https://item.jd.com/13482761.html
#                 @电子邮箱：fanmiao.cslt.thu@hotmail.com             
#                 @官方交流QQ群号：561500762                        
##################################################################

In [2]:
from tensorflow.keras import models, layers, losses, optimizers, Model
import tensorflow as tf

#设置超参数。
INPUT_UNITS = 56
TIME_STEPS = 14
NUM_HEADS = 8
HIDDEN_UNITS = 256
NUM_CLASSES = 10
EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 1e-3


class Transformer_Encoder(Model):
    '''
    自定义Transformer（编码器）模型类，继承自keras.Model。
    '''
    def __init__(self, input_units, time_steps, num_heads, hidden_units, num_classes):
        
        super(Transformer_Encoder, self).__init__()
        
        self.m_attn = layers.MultiHeadAttention(key_dim=input_units, num_heads=num_heads)
        
        self.ln = layers.LayerNormalization()
        
        self.l1 = layers.Dense(input_units, activation='relu')
        
        self.l2 = layers.Dense(hidden_units, activation='relu')
    
        self.l3 = layers.Dense(num_classes)


    def call(self, x):
        
        out_att = self.m_attn(x, x, x)
        
        out = self.ln(out_att + x)
        
        out_l1 = self.l1(out)
        
        out_l1 = self.relu(out_l1)
        
        out  = self.ln(out + out_l1)
        
        out = torch.reshape(out, (-1,784))
        
        out_l2 = self.l2(out)
        
        out_l2 = self.relu(out_l2)
        
        out_l3 = self.l3(out_l2)
        
        return out_l3 

    
#初始化Transformer（编码器）模型。
model = Transformer_Encoder(INPUT_UNITS, NUM_HEADS, NUM_CLASSES)

#设定神经网络的损失函数、优化方式，以及评估方法。
model.compile(optimizer=optimizers.Adam(LEARNING_RATE),
              loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

2023-02-20 17:23:08.917235: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TypeError: __init__() missing 2 required positional arguments: 'hidden_units' and 'num_classes'

In [ ]:
import pandas as pd


#使用pandas，读取fashion_mnist的训练和测试数据文件。
train_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_train.csv')
test_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_test.csv')

#从训练数据中，拆解出训练特征和类别标签。
X_train = train_data[train_data.columns[1:]]
y_train = train_data['label']

#从测试数据中，拆解出测试特征和类别标签。
X_test = test_data[train_data.columns[1:]]
y_test = test_data['label']

In [ ]:
from sklearn.preprocessing import StandardScaler


#初始化数据标准化处理器。
ss = StandardScaler()

#标准化训练数据特征。
X_train = ss.fit_transform(X_train)

#标准化测试数据特征。
X_test = ss.transform(X_test)

In [ ]:
X_train = X_train.reshape([-1, TIME_STEPS, INPUT_UNITS])

#使用fashion_mnist的训练集数据训练网络模型。
model.fit(X_train, y_train.values, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

In [ ]:
X_test = X_test.reshape([-1, TIME_STEPS, INPUT_UNITS])

#使用fashion_mnist的测试集数据评估网络模型的效果。
result = model.evaluate(X_test, y_test.values, verbose=0)

print('Transformer模型（TensorFlow版本）在fashion_mnist测试集上的准确率为: %.2f%%。' %(result[1] * 100))